In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
DATA_PATH = "/content/drive/MyDrive/final_pj/final_project_data/"
DATA_PATH

'/content/drive/MyDrive/final_pj/final_project_data/'

In [96]:
SEED = 42

In [113]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [114]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.0_1106.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.0_1106.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 1504), (12225, 1504))

In [99]:
train_ft.isnull().sum().sum()

0

In [100]:
test_ft.isnull().sum().sum()

0

In [115]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1503), (12225, 1503))

In [116]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류,22
주구매_중분류,246


In [103]:
cols

['주구매지점', '주구매_대분류', '주구매_중분류']

In [117]:
%pip install category_encoders

In [118]:
import category_encoders as ce
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[cols])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[cols])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1775), (12225, 1775))

In [106]:
cols

['주구매지점', '주구매_대분류', '주구매_중분류']

In [119]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1772), (12225, 1772))

In [120]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [121]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

In [122]:
scaler = MinMaxScaler()
scaler.fit(train_ft)
train_ft[train_ft.columns] = scaler.transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)

In [78]:
#scaler = StandardScaler()
#scaler.fit(train_ft)
#train_ft[train_ft.columns] = scaler.transform(train_ft)
#test_ft[test_ft.columns] = scaler.transform(test_ft)

In [79]:
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,주구매_중분류_237,주구매_중분류_238,주구매_중분류_239,주구매_중분류_240,주구매_중분류_241,주구매_중분류_242,주구매_중분류_243,주구매_중분류_244,주구매_중분류_245,주구매_중분류_246
0,-0.369867,0.002987,0.257728,-0.257728,-0.230862,-0.390544,-1.029777,0.001191,0.838272,0.338186,...,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182
1,0.144110,-0.356452,-1.008554,1.008554,-0.619841,0.412809,0.323951,-0.390607,0.620171,-0.552996,...,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182
2,1.943028,-0.869935,0.036742,-0.036742,1.616788,1.926821,0.798943,-0.514333,-0.304527,-0.059266,...,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182
3,3.793345,-1.023980,-0.080558,0.080558,3.172703,4.429574,0.420933,-0.327474,0.008592,-0.135636,...,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182
4,0.452496,-0.613193,0.302875,-0.302875,0.838830,0.567300,-0.752532,1.707410,-0.130285,-0.821561,...,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182,-0.008182,-0.008182,-0.011571,-0.008182,-0.008182


In [123]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


In [125]:
#min-max scaler
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')


params = {
    "random_state": 42,
    'n_jobs': -1,
    'n_estimators': 147,
    'num_leaves': 4,
    'min_child_samples': 15,
    'learning_rate': 0.4461496079568744,
    'colsample_bytree': 0.9643335778437738,
    'reg_alpha': 0.01724981316028688,
    'reg_lambda': 0.050648621080596416,
    'max_bin': 127,
    'verbose': -1
}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []
feature_importances = []

# Stratified K-Fold 교차 검증 설정
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]


    # 모델 초기화 및 학습
    model = LGBMClassifier(**params)
    model.fit(
    x_train, y_train,
    eval_set=[(x_valid, y_valid)],  # 검증 데이터 제공
    eval_metric="f1_macro"
    )

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

    feature_importances.append(model.feature_importances_)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7049154021320414


categorical_feature 추가
범주형 피처 최적화



In [93]:
#standardscaler
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')


params = {
    "random_state": 42,
    "learning_rate": 0.05,  # 학습률
    "n_estimators": 1000,   # 부스팅 단계 수
    "max_depth": 7,         # 최대 트리 깊이
    "num_leaves": 50,       # 리프 노드의 수 (예시 값)
    "min_child_samples": 20, # 최소 데이터 수 (예시 값)
    "subsample": 0.8,       # 샘플링 비율
    "colsample_bytree": 0.8, # 피처 샘플링 비율
    "reg_alpha": 0.1,       # L1 규제
    "reg_lambda": 0.1,      # L2 규제
    "min_child_weight": 3,  # (기존 값을 그대로 사용)
    "min_data_in_leaf": 20,  # (기존 값을 그대로 사용)
    "early_stopping_rounds": 20,
    "n_jobs": -1,
}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []
feature_importances = []

# Stratified K-Fold 교차 검증 설정
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]


    # 모델 초기화 및 학습
    model = LGBMClassifier(**params)
    model.fit(
    x_train, y_train,
    eval_set=[(x_valid, y_valid)],  # 검증 데이터 제공
    eval_metric="f1_macro"
    )

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

    feature_importances.append(model.feature_importances_)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] early_stopping_round is set=20, early_stopping_rounds=20 will be ignored. Current value: early_stopping_round=20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Number of positive: 4687, number of negative: 7265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.786480 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 127816
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 1450
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] early_stopping_round is set=20, early

In [112]:
#min_max_scaler
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')


params = {
    "random_state": 42,
    "learning_rate": 0.05,  # 학습률
    "n_estimators": 1000,   # 부스팅 단계 수
    "max_depth": 7,         # 최대 트리 깊이
    "num_leaves": 50,       # 리프 노드의 수 (예시 값)
    "min_child_samples": 20, # 최소 데이터 수 (예시 값)
    "subsample": 0.8,       # 샘플링 비율
    "colsample_bytree": 0.8, # 피처 샘플링 비율
    "reg_alpha": 0.1,       # L1 규제
    "reg_lambda": 0.1,      # L2 규제
    "min_child_weight": 3,  # (기존 값을 그대로 사용)
    "min_data_in_leaf": 20,  # (기존 값을 그대로 사용)
    "early_stopping_rounds": 20,
    "n_jobs": -1,
}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []
feature_importances = []

# Stratified K-Fold 교차 검증 설정
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]


    # 모델 초기화 및 학습
    model = LGBMClassifier(**params)
    model.fit(
    x_train, y_train,
    eval_set=[(x_valid, y_valid)],  # 검증 데이터 제공
    eval_metric="f1_macro"
    )

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

    feature_importances.append(model.feature_importances_)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] early_stopping_round is set=20, early_stopping_rounds=20 will be ignored. Current value: early_stopping_round=20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Number of positive: 4687, number of negative: 7265
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.295247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 133083
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 1450
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] early_stopping_round is set=20, early

In [91]:
feature_importances = np.mean(feature_importances, axis=0)  # 각 모델의 피처 중요도를 평균
feature_importance_df = pd.DataFrame({
    'Feature': train_ft.columns,  # 피처 이름
    'Importance': feature_importances  # 피처 중요도
})

# 중요도가 높은 순서대로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [92]:
feature_importance_df.head(20)

,Feature,Importance
18,18시_21시_구매비율,108.2
13,일별평균구매횟수,81.6
7,여름_구매비율,64.4
856,pv_중분류_수입종합화장품_구입금액비중,54.2
10,주구매요일,53.4
15,9시_12시_구매비율,44.2
889,pv_중분류_란제리_구입금액비중,42.8
220,pv_대분류_영플라자_구입금액비중,39.6
857,pv_중분류_아동_구입금액비중,39.0
108,평균구매액,38.0


In [ ]:
model = LGBMClassifier(**params)
model.fit(train_ft, target)

In [ ]:
pred = model.predict(test_ft)  # test_data는 테스트 데이터셋입니다.
pred

In [ ]:
submit["target"] = pred
submit

In [ ]:
submit.to_csv(f"{DATA_PATH}lgbm_1106_submit.csv",index=False)